In [1]:
import h5py    
import numpy as np  
import bluepysnap
import matplotlib.pyplot as plt
from mpl_toolkits.axisartist.axislines import Axes
import bluepysnap
from bluepysnap import Simulation as snap_sim
from pathlib import Path
# Importing Pandas to create DataFrame
import pandas as pd

In [2]:
def read_result(result_path):
    read_result = h5py.File(result_path)
    result = read_result['report']['All']['data']

    return(result)

def mapping_extract(result_path):
    read_result = h5py.File(result_path)
    result = read_result['report']['All']['data']
        
    extract_mapping = read_result['report']['All']['mapping']['node_ids'][:]
    
    t0 = read_result['report']['All']['mapping']['time'][0]
    T =read_result['report']['All']['mapping']['time'][1]
    dt = read_result['report']['All']['mapping']['time'][2]

    
    N= (T-t0)/dt
    time_line = np.linspace(t0,T, int(N))
    N2 = (T-t0)/0.025
    voltage_time_line = np.linspace(t0,T,int(N2))
    return(extract_mapping, time_line, voltage_time_line)

In [4]:
# Path where simulation results are stored
results_path ='/gpfs/bbp.cscs.ch/data/project/proj137/farinaNGV/Paper_results/my_simulation/'

In [3]:
# read the circuit from ../../config/circuit_config.json
# NOTE: in circuit_config.json many path are relative
# We need to read the circuit to get the info about the neurons as e-type and layer
#circuit = bluepysnap.Circuit('/gpfs/bbp.cscs.ch/data/project/proj137/farinaNGV/Paper_results/config/circuit_config.json')
circuit = bluepysnap.Circuit('../../config/circuit_config.json' )
node_population = circuit.nodes["All"]
pandas_nodes = node_population.get()
print('Nodes in the circuit', len(node_population.get().index.to_list()))

Nodes in the circuit 129348


In [5]:
sim_results_path = results_path + 'reporting_metabolism_young/'#  can be downloaded from 10.5281/zenodo.14187063

ATP_met = read_result(sim_results_path+  'ndam_atpi.h5')
after_ATP_met = read_result(sim_results_path+ 'metab_atpi_after_sync.h5')

mapping_met,time_line_met, volt_time_met = mapping_extract(sim_results_path+ 'ndam_nai.h5')

In [6]:
sim_results_path = results_path + 'reporting_metabolism_aged/'#  can be downloaded from 10.5281/zenodo.14187063


ATP_aged = read_result(sim_results_path+  'ndam_atpi.h5')
after_ATP_aged = read_result(sim_results_path+ 'metab_atpi_after_sync.h5')

mapping_aged,time_line_aged, volt_time_aged = mapping_extract(sim_results_path+ 'ndam_nai.h5')

In [7]:
# Importing Pandas to create DataFrame
import pandas as pd
 
# Creating Empty DataFrame and Storing it in variable df
df = pd.DataFrame()

In [8]:
df['ids'] = mapping_met
df['etype'] = [pandas_nodes['etype'][i] for i in df['ids']]
df['layer'] = [pandas_nodes['layer'][i] for i in df['ids']]
df['synapse_class'] = [ pandas_nodes['synapse_class'][i] for i in df['ids']]
df['x'] = [pandas_nodes['x'][i] for i in df['ids']]
df['y'] = [pandas_nodes['y'][i] for i in df['ids']]
df['z'] = [pandas_nodes['z'][i] for i in df['ids']]
df['final avg ATP'] = [after_ATP_met[-1,i] for i in range(len(df['ids']))]

In [12]:
df['final avg ATP aged'] = [after_ATP_aged[-1,i] for i in range(len(df['ids']))]

In [13]:
df['ATP difference'] = abs(df['final avg ATP'] - df['final avg ATP aged'])

In [9]:
#df.to_csv('../Data/ATP_difference_young_aged.csv', index=False)

,ids,etype,layer,synapse_class,x,y,z,final avg ATP
0,15,bNAC,1,INH,2084.624393,1059.532151,-683.156252,1.295010
1,9688,cADpyr,3,EXC,2152.568740,644.909169,-928.011106,1.312492
2,19377,cNAC,3,INH,2124.534901,977.777711,-902.898927,1.318045
3,29550,cADpyr,6,EXC,2311.046658,1033.548804,-1922.301593,1.277863
4,40058,cADpyr,6,EXC,2154.322095,1035.741302,-1914.334943,1.282802
...,...,...,...,...,...,...,...,...
27957,87146,cADpyr,2,EXC,2091.040827,657.538143,-799.599805,1.314256
27958,96387,dSTUT,5,INH,2347.780136,1120.580718,-1767.094450,1.274112
27959,105473,cADpyr,5,EXC,2208.193833,850.692110,-1673.365599,1.310398
27960,114402,cADpyr,5,EXC,2202.135787,922.822707,-1590.891189,1.310770
